In [ ]:
#!pip install advertools

In [ ]:
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import el_core_news_sm
from spacy.lang.el import GreekLemmatizer
from spacy.lang.el import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES


In [ ]:
df = pd.read_csv(r'E:\Various\Data Science\ToTranslate\Open_Pct_Per_Subject_trn.csv',sep = ";")

In [ ]:
nlp = el_core_news_sm.load()

In [ ]:
sw = nlp.Defaults.stop_words

In [ ]:
lemmatizer = GreekLemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

In [ ]:
def remove_ton(text):
    diction = {'ά':'α','έ':'ε','ί':'ι','ό':'ο','ώ':'ω','ύ':'υ'}
    for key in diction.keys():
        text = text.lower().replace(key, diction[key])
    return text    

In [ ]:
def clean_text(text):
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove stop words
    text = [x for x in text if x not in sw]
    # lower text
    text = [remove_ton(x) for x in text]
    #remove quotes
    text = [x.replace('quot;','').replace('&quot','') for x in text if x not in ['quot','amp']]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove amp & quot
    text = [x for x in text if x not in ['quot','amp']]
    # remove words with only one letter
    text = " ".join([t for t in text if len(t) > 1])
    # lemmatize text
    text = " ".join([lemmatizer(t.text,t.pos_)[0] for t in nlp(text)])
    return(text)

In [ ]:
df = df[['AA','SubjectLine','Pct','count_Sent']]

In [ ]:
df['SubjectLine_clean'] = df['SubjectLine'].apply(lambda x : clean_text(x) )

In [ ]:
df.head()

In [ ]:
tfidf = TfidfVectorizer(min_df = 10,ngram_range = (1,2))


In [ ]:
tfidf_result = tfidf.fit_transform(df['SubjectLine_clean']).toarray()


In [ ]:
tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())

In [ ]:
tfidf_df.columns = [str(x) for x in tfidf_df.columns]


In [ ]:
tfidf_df.index = df.index


In [ ]:
df = pd.concat([df, tfidf_df], axis=1)

In [ ]:
df.head()

In [ ]:
df.to_excel(r'E:\Various\Data Science\ToTranslate\Open_Pct_Per_Subject_SpaCy.xlsx')